# GPX Viewer

This app lets you to display a track from a GPX file recorded with a GPS device.

Built with [voila](https://github.com/QuantStack/voila), [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet), [bqplot](https://github.com/bloomberg/bqplot), [gpxpy](https://github.com/tkrajina/gpxpy).

In [ ]:
import datetime
import json
from collections import defaultdict
from io import StringIO
from statistics import mean

import gpxpy
import srtm
import pandas as pd

from bqplot import Axis, Figure, Lines, LinearScale
from bqplot.interacts import IndexSelector
from ipyleaflet import FullScreenControl, Map, MeasureControl, Polyline, Marker, CircleMarker
from ipywidgets import HTML, HBox, Label, Output, IntSlider, Layout

In [ ]:
def parse_data(file):
    gpx = gpxpy.parse(file)
    elevation_data = srtm.get_data()
    elevation_data.add_elevations(gpx, smooth=True)
    return gpx

In [ ]:
def plot_map(gpx):
    points = [p.point for p in gpx.get_points_data(distance_2d=True)]
    mean_lat = mean(p.latitude for p in points)
    mean_lng = mean(p.longitude for p in points)

    # create the map
    m = Map(center=(mean_lat, mean_lng), zoom=10)

    # show trace
    line = Polyline(locations=[[[p.latitude, p.longitude] for p in points],],
                    color = "red", fill=False)
    m.add_layer(line)

    # add markers
    for point in gpx.waypoints:
        lat, lng = point.latitude, point.longitude
        marker = Marker(location=(lat, lng), title=point.name, popup=HTML(value=point.name), draggable=False)
        m.add_layer(marker)

    # enable full screen mode
    m.add_control(FullScreenControl())
        
    # add measure control
    measure = MeasureControl(
        position='bottomleft',
        active_color = 'orange',
        primary_length_unit = 'kilometers'
    )
    m.add_control(measure)
        
    return m

In [ ]:
def plot_stats(gpx):
    lowest, highest = gpx.get_elevation_extremes()
    uphill, downhill = gpx.get_uphill_downhill()
    points = gpx.get_points_data(distance_2d=True)
    
    _, distance_from_start, *rest = points[-1]
    
    stats = defaultdict(list)
    stats['Date'].append(gpx.get_time_bounds().start_time.strftime("%Y-%m-%d"))
    stats['Distance'].append(round(distance_from_start / 1000, 2))
    stats['Duration'].append(str(datetime.timedelta(seconds=gpx.get_duration())))
    stats['Lowest'].append(int(lowest))
    stats['Highest'].append(int(highest))
    stats['Uphill'].append(int(uphill))
    stats['Downhill'].append(int(downhill))
    
    df = pd.DataFrame(
        stats,
        columns=['Date', 'Distance', 'Duration', 'Lowest', 'Highest', 'Uphill', 'Downhill']
    )
    
    return HTML(value=df.to_html(index=False),
               layout=Layout(width='100%', grid_area='header'))

In [ ]:
def plot_elevation(gpx):
    """
    Return an elevation graph for the given gpx trace
    """
    points = gpx.get_points_data(distance_2d=True)
    px = [p.distance_from_start / 1000 for p in points]
    py = [p.point.elevation for p in points]
    
    x_scale, y_scale = LinearScale(), LinearScale()
    x_ax = Axis(label='Distance (km)', scale=x_scale)
    y_ax = Axis(label='Elevation (m)', scale=y_scale, orientation='vertical')
    
    lines = Lines(x=px, y=py, scales={'x': x_scale, 'y': y_scale})
    
    elevation = Figure(title='Elevation Chart', axes=[x_ax, y_ax], marks=[lines])
    elevation.layout.width = 'auto'
    elevation.layout.height = 'auto'
    elevation.layout.min_height = '500px'

    elevation.interaction = IndexSelector(scale=x_scale)
    
    return elevation

In [ ]:
def link_trace_elevation(trace, elevation, gpx, debug):
    """
    Link the trace the elevation graph.
    Changing the selection on the elevation will update the
    marker on the map
    """
    points = gpx.get_points_data(distance_2d=True)
    _, distance_from_start, *rest = points[-1]
    n_points = len(points)
    
    def find_point(distance):
        """
        Find a point given the distance
        """
        progress = min(1, max(0, distance / distance_from_start))
        position = int(progress * (n_points - 1))
        return points[position].point
    
    # mark the current position on the map
    start = find_point(0)
    marker = CircleMarker(visible=False, location=(start.latitude, start.longitude),
                          radius=10, color="green", fill_color="green")
    trace.add_layer(marker)
    
    brushintsel = elevation.interaction
    
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected * 1000  # convert from km to m
        point = find_point(selected)
        marker.location = (point.latitude, point.longitude)
        
        position = max(0, int((selected / distance_from_start) * len(points)))
        debug.value = str(position) + " " + str(distance_from_start)
        
    brushintsel.observe(update_range, 'selected')

In [ ]:
with open('./trace.gpx') as f:
    gpx = parse_data(f)
    
    trace = plot_map(gpx)
    elevation = plot_elevation(gpx)
    debug = Label(value='')
    
    display(trace)
    display(elevation)
    display(debug)
    
    link_trace_elevation(trace, elevation, gpx, debug)